# Classificador

# Análise geral

Para a realização do projeto, foi escolhido pela dupla como tema a ser analisado a marca chinesa Xiaomi, que está ganhando espaço no mercado atualmente, sendo comparada com outras marcas, como a Apple, e gerou dois grupos, divididos em os que são a favor dos aparelhos Xiaomi e os que são contra, desses dois grupos muitos publicam no Twitter as suas opiniões. Então através da conta developer do Twitter, foi possível obter tweets sobre esse assunto, sendo 330 de treinamento e 200 de teste, primeiramente, todos foram classificados a mão, obtendo o número 0 aqueles que eram a favor da marca, ou seja, aqueles que defendiam, que falavam que os produtos da marca tem alta qualidade. Já o número 1 foi atribuído a tweets que criticavam a marca, criticavam aqueles que estão adquirindo os produtos da Xiaomi. Com essa base de dados, foi feito o classificador, treinando-o a partir dos 330 tweets já classificados, aplicando Naives Bayes.

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import re
from IPython.display import display
pd.options.display.max_rows = 13

Limpeza simples:

In [2]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[\/!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

## Análise dos tweets marca Xiaomi

A análise foi baseada na classificação dos tweets, de forma binária, sendo favoráveis a marca (0), ou contra a marca (1)

In [3]:
import os

os.getcwd()

'C:\\Users\\User\\Desktop\\2ºsemestre\\Ciência dos dados\\P2_tweets'

In [4]:
tweets = pd.read_excel('xiaomi_1.xlsx')

In [5]:
tweets;

In [6]:
#apagar tweets que foram repetidos ('RTs')
n_repete_tt = tweets.drop_duplicates()
n_repete_tt;

In [7]:
dict_n_rep_tt= n_repete_tt.to_dict('list')

In [8]:
dict_n_rep_tt

{'Treinamento': ['@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade, o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema. ela é pesada, lenta e com várias falhas. a xiaomi por exemplo, usa a miui, uma interface madura e bem otimizada.',
  'esse cel xiaomi é excelente! 😁',
  '@hugorambo18 não temos vagas, abandonou a família xiaomi',
  '@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais, custavam quase 200 cada, no brasil, paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas.',
  '@itsramss aqui e team xiaomi ramenex junte-se a nós',
  'xiaomi eh a o anti cristo não caiam nessa https://t.co/ctkzhk3wlq',
  'mano, nem tô acreditando que depois de indicar pra todo mundo, eu finalmente virei testemunha de xiaomi 😂😂😂  inclusive, tô amando meu rn7 @xiaomibrasil. continuarei propagando ao mundo teus feitos 😂😂😂♥️',
  '@grrvide1 me sinto uma 

In [9]:
lista_tt = dict_n_rep_tt['Treinamento']
lista_tt

['@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade, o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema. ela é pesada, lenta e com várias falhas. a xiaomi por exemplo, usa a miui, uma interface madura e bem otimizada.',
 'esse cel xiaomi é excelente! 😁',
 '@hugorambo18 não temos vagas, abandonou a família xiaomi',
 '@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais, custavam quase 200 cada, no brasil, paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas.',
 '@itsramss aqui e team xiaomi ramenex junte-se a nós',
 'xiaomi eh a o anti cristo não caiam nessa https://t.co/ctkzhk3wlq',
 'mano, nem tô acreditando que depois de indicar pra todo mundo, eu finalmente virei testemunha de xiaomi 😂😂😂  inclusive, tô amando meu rn7 @xiaomibrasil. continuarei propagando ao mundo teus feitos 😂😂😂♥️',
 '@grrvide1 me sinto uma testemunha do xiaomi ok

In [10]:
lista_classificacao = dict_n_rep_tt['Classificação']

In [11]:
i=0
lista_sem_caracteres = []
for i in range(len(lista_tt)):
    novo_tt = cleanup(lista_tt[i])
    lista_sem_caracteres.append(novo_tt.lower())
    i += 1

In [12]:
lista_sem_caracteres

['@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade  o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema  ela é pesada  lenta e com várias falhas  a xiaomi por exemplo  usa a miui  uma interface madura e bem otimizada ',
 'esse cel xiaomi é excelente  😁',
 '@hugorambo18 não temos vagas  abandonou a família xiaomi',
 '@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais  custavam quase 200 cada  no brasil  paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas ',
 '@itsramss aqui e team xiaomi ramenex junte se a nós',
 'xiaomi eh a o anti cristo não caiam nessa https   t co ctkzhk3wlq',
 'mano  nem tô acreditando que depois de indicar pra todo mundo  eu finalmente virei testemunha de xiaomi 😂😂😂  inclusive  tô amando meu rn7 @xiaomibrasil  continuarei propagando ao mundo teus feitos 😂😂😂♥️',
 '@grrvide1 me sinto uma testemunha do xiaomi ok

In [13]:
dict_df = {}
    
dict_df['Treinamento'] = lista_sem_caracteres
dict_df['Classificação'] = lista_classificacao

In [14]:
dict_df

{'Treinamento': ['@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade  o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema  ela é pesada  lenta e com várias falhas  a xiaomi por exemplo  usa a miui  uma interface madura e bem otimizada ',
  'esse cel xiaomi é excelente  😁',
  '@hugorambo18 não temos vagas  abandonou a família xiaomi',
  '@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais  custavam quase 200 cada  no brasil  paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas ',
  '@itsramss aqui e team xiaomi ramenex junte se a nós',
  'xiaomi eh a o anti cristo não caiam nessa https   t co ctkzhk3wlq',
  'mano  nem tô acreditando que depois de indicar pra todo mundo  eu finalmente virei testemunha de xiaomi 😂😂😂  inclusive  tô amando meu rn7 @xiaomibrasil  continuarei propagando ao mundo teus feitos 😂😂😂♥️',
  '@grrvide1 me sinto uma 

In [15]:
#novo banco de dados apos limpeza simples dos tweets
novo_tweets = pd.DataFrame(data=dict_df)
novo_tweets

,Treinamento,Classificação
0,@marcusc13 @xiaomibrasil @apple @motorolabr @s...,0
1,esse cel xiaomi é excelente 😁,0
2,@hugorambo18 não temos vagas abandonou a famí...,1
3,@alysonhoffmann comprei 2 fones da xiaomi no d...,0
4,@itsramss aqui e team xiaomi ramenex junte se ...,0
5,xiaomi eh a o anti cristo não caiam nessa http...,1
...,...,...
321,fui pesquisar sobre a xiaomi e gostei sla ache...,0
322,isso demonstra que muitos equipamentos são de ...,0
323,@mumudesp se quiser eu pego meus slides e te a...,0


In [16]:
#analise dos emojis
emoji_count = defaultdict(int)
for i in novo_tweets['Treinamento']:
    for emoji in re.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]', i):
        emoji_count[emoji] += 1
emoji_count;

In [17]:
lista_sem_caracteres = dict_n_rep_tt['Treinamento']

Tweest a favor da marca:

In [18]:
afavor_tweets= novo_tweets.loc[(novo_tweets.Classificação==0)]

In [19]:
afavor_tweets

,Treinamento,Classificação
0,@marcusc13 @xiaomibrasil @apple @motorolabr @s...,0
1,esse cel xiaomi é excelente 😁,0
3,@alysonhoffmann comprei 2 fones da xiaomi no d...,0
4,@itsramss aqui e team xiaomi ramenex junte se ...,0
6,mano nem tô acreditando que depois de indicar...,0
7,@grrvide1 me sinto uma testemunha do xiaomi ok...,0
...,...,...
321,fui pesquisar sobre a xiaomi e gostei sla ache...,0
322,isso demonstra que muitos equipamentos são de ...,0
323,@mumudesp se quiser eu pego meus slides e te a...,0


In [20]:
n_afavor_tweets= novo_tweets.loc[(novo_tweets.Classificação==1)]

In [21]:
n_afavor_tweets

,Treinamento,Classificação
2,@hugorambo18 não temos vagas abandonou a famí...,1
5,xiaomi eh a o anti cristo não caiam nessa http...,1
12,rt @treviagabriel amiga n beije este homem el...,1
16,crossfiteiro é mais chato que fã da xiaomi,1
17,vou comprar meu xiaomi se eu tiver chorando é ...,1
19,será que esses xiaomi são bons mesmo,1
...,...,...
291,@guidofraga @matosrs_ tem um vídeo do loop inf...,1
301,e um fone xiaomi tb pq esse inferno de celular...,1
303,xiaomi lovers é quase uma religião ficam tenta...,1


In [22]:
# Analise bem crua dos tweets
P_1 = n_afavor_tweets['Treinamento'].count()/ (n_afavor_tweets['Treinamento'].count() + afavor_tweets['Treinamento'].count())
P_0 = 1-P_1

In [23]:
#Probabilidade de gostarem do produto Xiaomi; p(0)= 0,74
P_0

0.7400611620795108

In [24]:
# Probabilidade de não gostarem do produto Xiaomi; p(1)= 0,26
P_1

0.2599388379204893

In [25]:
#serie_afavor= pd.Series(afavor_tweets['Treinamento'])

In [26]:
#serie_afavor

In [27]:
#afavor_frel= [serie_afavor.spilt()].prod()

In [28]:
# Analise de quem é a favor da Xiaomi
txt_afavor=''
for t in afavor_tweets['Treinamento']:
    t=str(t)
    txt_afavor+=t

In [29]:
afavor=pd.Series(txt_afavor.split())

In [30]:
afavor

0           @marcusc13
1        @xiaomibrasil
2               @apple
3          @motorolabr
4       @samsungbrasil
5                   na
             ...      
4167      condicionado
4168                da
4169              sala
4170               com
4171               ele
4172               🤩🤩😂
Length: 4173, dtype: object

In [31]:
#frequencias absolutas
tabela_afavor = afavor.value_counts()
tabela_afavor

xiaomi            201
o                  96
que                87
de                 87
e                  82
um                 77
                 ... 
😂😂😂♥️@grrvide1      1
mandei              1
atualmente          1
128gbtop            1
enaltecer           1
boy                 1
Length: 1462, dtype: int64

In [32]:
#palavras no grupo que são a favor sem que haja repetição
lista_afavor = []
for palavra in tabela_afavor.index:
    lista_afavor.append(palavra)
len(lista_afavor) #será usado na suavização de Laplace
#lista_afavor.count('xiaomi')

1462

In [33]:
#frequencias relativas
tabela_afavor_relativa = afavor.value_counts(True)

In [34]:
tabela_afavor_relativa

xiaomi            0.048167
o                 0.023005
que               0.020848
de                0.020848
e                 0.019650
um                0.018452
                    ...   
😂😂😂♥️@grrvide1    0.000240
mandei            0.000240
atualmente        0.000240
128gbtop          0.000240
enaltecer         0.000240
boy               0.000240
Length: 1462, dtype: float64

In [35]:
freq_afavor_min = tabela_afavor_relativa.min()

In [36]:
# Analise de quem é contra a Xiaomi
txt_contra=''
for e in n_afavor_tweets['Treinamento']:
    e=str(e)
    txt_contra+=e

In [37]:
contra=pd.Series(txt_contra.split())

In [38]:
#frequencias absolutas
tabela_n_afavor = contra.value_counts()
tabela_n_afavor

xiaomi              71
de                  25
a                   24
que                 23
um                  22
o                   19
                    ..
virou                1
diego                1
xiaomiredminote7     1
n°                   1
extremamente         1
jales                1
Length: 531, dtype: int64

In [39]:
#palavras no grupo que são contra sem que haja repetição
lista_contra = []
for palavra in tabela_n_afavor.index:
    lista_contra.append(palavra)
len(lista_contra) #será usado na suavização de Laplace

531

In [40]:
#frequencias relativas
tabela_contra_relativa = contra.value_counts(True)

In [41]:
tabela_contra_relativa

xiaomi              0.067943
de                  0.023923
a                   0.022967
que                 0.022010
um                  0.021053
o                   0.018182
                      ...   
virou               0.000957
diego               0.000957
xiaomiredminote7    0.000957
n°                  0.000957
extremamente        0.000957
jales               0.000957
Length: 531, dtype: float64

In [42]:
freq_contra_min = tabela_contra_relativa.min()

In [43]:
# Assuma que a soma das palavras do texto a favor e do texto contra
#formam o meu conjunto universo para analise 

todas_palavras = txt_afavor + txt_contra
todas_palavras;

In [44]:
palavras = pd.Series(todas_palavras.split())

# todas as palavras contidas no universo que estamos trabalhando
lista_palavras_treinamento = []
for palavra in palavras:
    lista_palavras_treinamento.append(palavra)

In [45]:
# Probabilidade de cada palavra aparecer no espaço amostral (contra + a favor) que estamos trabalhando
tabela_palavras_relativa = palavras.value_counts(True)
tabela_palavras_relativa

xiaomi    0.052137
o         0.022043
de        0.021468
que       0.021085
e         0.019360
um        0.018976
            ...   
essas     0.000192
massa     0.000192
aquele    0.000192
huawei    0.000192
fofo      0.000192
app       0.000192
Length: 1769, dtype: float64

In [46]:
set_afavor = set(tabela_afavor_relativa.index)
set_n_afavor = set(tabela_contra_relativa.index)

inter = list(set_afavor.intersection(set_n_afavor))

In [47]:
# Palavras que aparecem tanto no conjunto que é a favor da Xiaomi quanto palavras que estão no conjunto que não é a favor do produto.
inter;

In [48]:
# Essa frase pode ser substituida por qualquer outra. Ela está aí apenas a titulo de estudo mesmo.
# Vamos fazer com que cada tweet de teste seja uma 'frase'
frase = "xiaomi conquistou o meu coracao"
frase

'xiaomi conquistou o meu coracao'

In [49]:
lista_palavras = frase.split()

In [50]:
#calculo do numero de palavras que existem em ambas as categorias de tweets
universo = palavras.count()
universo

5217

In [51]:
# Probabilidade de gostar --> P(0)

total_afavor = tabela_afavor.sum()
P_0 = total_afavor/universo
P_0

0.7998849913743531

In [52]:
# Probabilidade de NÃO gostar --> P(1)

total_n_afavor = tabela_n_afavor.sum()
P_1 = total_n_afavor/universo
P_1

0.2003066896683918

In [53]:
# Probabilidade da frase dado que não gosta do produto
P_frase_ngosta = tabela_contra_relativa[lista_palavras].prod()
P_frase_ngosta

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\series.py:851: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


8.274868832090507e-06

In [72]:
tabela_contra_relativa[lista_palavras]

xiaomi        0.067943
conquistou         NaN
o             0.018182
meu           0.006699
coracao            NaN
dtype: float64

In [54]:
# Calculo da probabilidade da frase dado que gosta do produto
P_frase_gosta = tabela_afavor_relativa[lista_palavras].prod()
P_frase_gosta

1.3807831542832243e-05

In [55]:
# Probabilidade de gostar dado que temos a frase
P_gosta_frase = P_frase_gosta*P_0
P_gosta_frase

1.104467721453689e-05

In [56]:
# Probabilidade de NÃO gostar dado que temos a frase:
P_ngosta_frase = P_frase_ngosta*P_1
P_ngosta_frase

1.6575115831962008e-06

Agora vamos testar os tweets 

In [57]:
#Começando o teste

testando = pd.read_excel('teste.xlsx')

In [58]:
testando

,Teste
0,xiaomi é o celular que emite mais radiação só ...
1,kkkkkkkkkkk os fanboys de xiaomi espumando no ...
2,dei um tombo no meu celular ontem na plataform...
3,@sabrina27114057 compra um xiaomi
4,@babiie_ @olhardigital acabei de pegar um xiao...
5,olhem lá se eu usasse a marca do meu telemóvel...
...,...
194,@xiaomi_br @brunnarangel foi indicada ao emmy ...
195,🍃🌧️ já é quase sexta-feira... \n\n#mi9t\n#xiao...
196,cabo de dados e carga rápida blitzwolf 2 in 1 ...


In [59]:
#Limpeza 

lista_teste=[]
for t in testando['Teste']:
    t_limpo = cleanup(t.lower())
    lista_teste.append(t_limpo)
lista_teste;

In [60]:
lista_palavras_teste = []
for tweet in lista_teste:
    lista_palavras_teste.append(tweet.split())
    

In [61]:
lista_palavras_teste;

In [62]:
#analise dos emojis
emoji_count_teste = defaultdict(int)
for i in testando['Teste']:
    for emoji in re.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]', i):
        emoji_count_teste[emoji] += 1
emoji_count_teste;

In [75]:
teste_classificacao=[]
i = 0
while i < len(lista_palavras_teste):
    #pega cada lista da lista de teste e compara com a lista de treinamento (conjunto universo)
    conjunto_teste = set(lista_palavras_teste[i])
    conjunto_treinamento_afavor = set(lista_contra)
    conjunto_treinamento_contra = set(lista_afavor)
    # queremos a interseção dos conjuntos. Ou seja, todas as palavras que aparecem em teste, devem aparecer no treinamento. 
    # Para as que não aparecerem, iremos fazer a suavização de Laplace
    inter_buscada_afavor = list(conjunto_teste.intersection(conjunto_treinamento_afavor))
    inter_buscada_contra = list(conjunto_teste.intersection(conjunto_treinamento_contra))
    
    
    #tudo oq tem nos tweets de teste e nao tem na interseção
    diferenca_afavor = list(conjunto_teste.difference(set(inter_buscada_afavor)))
    diferenca_contra = list(conjunto_teste.difference(set(inter_buscada_contra)))
     
    if len(diferenca_afavor)==0: #significa que todas as palavras de teste estão presentes no treinamento
        P_tweet_gosta = tabela_afavor_relativa[lista_palavras_teste[i]].prod()
       #P_gosta_tweet = P_tweet_gosta*P_0
    
    else:
        prod_afavor = ((freq_afavor_min/2)**len(diferenca_afavor))
        P_tweet_gosta = (tabela_afavor_relativa[inter_buscada_afavor].prod())*(prod_afavor)
    
    if len(diferenca_contra)==0:
        P_tweet_ngosta = tabela_contra_relativa[lista_palavras_teste[i]].prod()
        #P_ngosta_tweet = P_tweet_ngosta*P_1
    else:
        #utilizacao do conceito de frequencia minima. A frequencia das palavras que nao aparecem na intersecção sera calculada como
        # a (frequencia minima do grupo)/2
        #prod_afavor = ((freq_afavor_min/2)**len(diferenca))
        #P_tweet_gosta = (tabela_afavor_relativa[inter_buscada].prod())*(prod_afavor)
        
        prod_contra = ((freq_contra_min/2)**len(diferenca_contra))
        P_tweet_ngosta = tabela_contra_relativa[inter_buscada_contra].prod()*(prod_contra)
            
    P_gosta_tweet = P_tweet_gosta*P_0
    P_ngosta_tweet = P_tweet_ngosta*P_1
    
    if P_gosta_tweet>P_ngosta_tweet:
        teste_classificacao.append(0)
    else: 
        teste_classificacao.append(1) 
    
    i+=1          
            

In [76]:
teste_classificacao

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [65]:
dict_teste = {}
    
dict_teste['Teste'] = lista_teste
dict_teste['Classificação'] = teste_classificacao

In [68]:
classificados = pd.DataFrame(data=dict_teste)
classificados

,Teste,Classificação
0,xiaomi é o celular que emite mais radiação só ...,1
1,kkkkkkkkkkk os fanboys de xiaomi espumando no ...,1
2,dei um tombo no meu celular ontem na plataform...,1
3,@sabrina27114057 compra um xiaomi,1
4,@babiie_ @olhardigital acabei de pegar um xiao...,1
5,olhem lá se eu usasse a marca do meu telemóvel...,1
...,...,...
194,@xiaomi_br @brunnarangel foi indicada ao emmy ...,1
195,🍃🌧️ já é quase sexta feira \n\n mi9t\n xiao...,1
196,cabo de dados e carga rápida blitzwolf 2 in 1 ...,1


## Comparando